### Importing the necessary libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd

### Getting the html source code using the requests library

In [2]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

### Extracting the source code into a Beautiful Soup Library's object , named here --> "soup"

In [3]:
soup = BeautifulSoup(source, 'lxml')

### Saving the table from the website using the find function in a variable called "table"

In [4]:
table = soup.find('table')

### Processing the variable containing the table from the website and storing it in a dataframe

In [5]:
column_names = ['Postal Code', 'Borough', 'Neighbourhood']
tr_tags = table.find_all('tr')
i = 1
table_data = list()
for tr in tr_tags:
    if i == 1:
        i += 1
    else:
        row = list()
        td_tags = tr.find_all('td')
        for td in td_tags:
            text = td.text.strip()
            row.append(text)
            
        if row[1] != "Not assigned":
            
            table_data.append(row)
                
dataframe = pd.DataFrame(table_data, columns=column_names)
dataframe.to_csv("toronto_table.csv")
dataframe

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [6]:
dataframe.shape

(103, 3)

### Opening the csv file containing the latitudes and longitudes and storing in a pandas dataframe, here named "data"

In [7]:
with open('Geospatial_Coordinates.csv') as file:
    data = pd.read_csv(file)
    
data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


### Merging the two dataframes in a single one

In [18]:
dataframe_merged = pd.merge(dataframe, data, on='Postal Code')
dataframe_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [9]:
dataframe_merged.shape

(103, 5)

### Importing further required libraies to explore the data and map the locations of the neighbourhoods

In [10]:
import numpy as np

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



### Fetching the coordinates of 'Toronto' using geopy library

In [19]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Tornoto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Tornoto are 43.6534817, -79.3839347.


### Mapping the neighbourhoods in the dataset using the Folium library

In [20]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, dataframe_merged in zip(dataframe_merged['Latitude'], dataframe_merged['Longitude'], dataframe_merged['Borough'], dataframe_merged['Neighbourhood']):
    label = '{}, {}'.format(dataframe_merged, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto